In [3]:
from z3 import *

# ============================ 共用工具 ============================

def z3_to_float(z3_val):
    """Z3 數值轉 Python float，若為無限小數用 as_decimal() 截取。"""
    try:
        dec_str = z3_val.as_decimal(20)
        if dec_str.endswith('?'):
            dec_str = dec_str[:-1]
        return float(dec_str)
    except Exception:
        return float(str(z3_val))

# =================================================================
# 高額消費貨物 / 勞務稅  (新增 free_vars 版)
# =================================================================

# ------------------ 高額消費貨物稅 ------------------

def calculate_high_consumption_goods_tax(
    item_type: str,
    base_amount: float,
    *,
    free_vars: list | None = None,
):
    """以 10% 稅率計算高額消費貨物稅，並可最小化 base_amount。"""
    if free_vars is None:
        free_vars = []
    free_vars = set(free_vars)

    opt = Optimize()

    ba_z = Real("ba")
    opt.add(ba_z >= 0)
    if "base_amount" not in free_vars:
        opt.add(ba_z == base_amount)

    tax_expr = ba_z * 0.10
    tax_int = Int("tax_int")
    opt.add(tax_int == ToInt(tax_expr), tax_int >= 0)
    opt.minimize(tax_int)

    if opt.check() != sat:
        return None

    best_tax = opt.model()[tax_int].as_long()
    return best_tax  # 回傳整數元

# ------------------ 高額消費勞務稅 ------------------

def calculate_high_consumption_services_tax(
    sales_price: float,
    *,
    free_vars: list | None = None,
):
    """以 10% 稅率計算高額消費勞務稅，並可最小化 sales_price。"""
    if free_vars is None:
        free_vars = []
    free_vars = set(free_vars)

    opt = Optimize()

    sp_z = Real("sp")
    opt.add(sp_z >= 0)
    if "sales_price" not in free_vars:
        opt.add(sp_z == sales_price)

    tax_expr = sp_z * 0.10
    tax_int = Int("tax_int")
    opt.add(tax_int == ToInt(tax_expr), tax_int >= 0)
    opt.minimize(tax_int)

    if opt.check() != sat:
        return None

    return opt.model()[tax_int].as_long()

# =================================================================
# Demo 區：直接執行此檔可看到差異
# =================================================================
if __name__ == "__main__":
    print("\n=== 高額消費貨物稅 ===")
    print("固定:", calculate_high_consumption_goods_tax("小客車", 3_000_000))
    print("base_amount free:", calculate_high_consumption_goods_tax("小客車", 3_000_000, free_vars=["base_amount"]))

    print("\n=== 高額消費勞務稅 ===")
    print("固定:", calculate_high_consumption_services_tax(500_000))
    print("sales_price free:", calculate_high_consumption_services_tax(500_000, free_vars=["sales_price"]))



=== 高額消費貨物稅 ===
固定: 300000
base_amount free: 0

=== 高額消費勞務稅 ===
固定: 50000
sales_price free: 0
